In [19]:
# Edited by Weiyang on 01/22/2019
import xarray as xr
import numpy as np
#np.set_printoptions(threshold=np.nan)
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
#import gsw
import os

In [15]:
# Define a function to calculate volume flux above the sill crest
def Vflux(filename, sill_depth):
    
    from scipy.interpolate import interp2d
    
    state = xr.open_dataset(filename)
    # Grid info
    Zgrid = int(sill_depth / 3)
    grid_size = 200 * 3
    y = state.Y.data[1:10]
    z0 = state.Z.data
    z1 = np.arange(-1.5,-150,-3)
    
    Qs_out = np.empty(len(state.T))
    Qs_in = np.empty(len(state.T))
    for t in range(len(state.T)):
        u = state.U.isel(T = t)        
        u_sill = (u.data[:, 1:10, 29]) # Profile above the sill crest
        f_u = interp2d(y, z0, u_sill) # Interpolate in Z direction
        u_sill_interp = f_u(y, z1)
        Qs = u_sill_interp[-Zgrid:,:] * grid_size
        Qs_out[t] = Qs[Qs > 0].sum()
        Qs_in[t] = -Qs[Qs < 0].sum()
        
    return (Qs_out, Qs_in)

In [20]:

fig_path = "/Users/weiyangbao/Documents/Jupyter/Vol_flux"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)
    
import glob
files = glob.glob('/Users/weiyangbao/Desktop/sill12/state_*.nc')
sill_depth = 12

for i in range(len(files)):
    Qs_out, Qs_in = Vflux(files[i], sill_depth)
    #create figure
    fig, ax =plt.subplots(figsize=(12,6))
    ax.set_xlim([0,240])
    ax.set_xticks(np.linspace(0,240,13))
    ax.set_xticklabels(np.linspace(0,120,13, dtype = int), size = 20)
    plt.yticks(fontsize = 20)
    plt.xlabel("Time [Day]", size = 20, weight = 'bold')
    plt.ylabel("Volume Flux [m\u00B3/s]", size = 20, weight = 'bold')
    #plt.tick_params(axis="y", labelcolor="b")
    plt.plot(Qs_out, "b-", linewidth=1.5, label='Q_out')
    plt.plot(Qs_in, "r-", linewidth=1.5, label='Q_in')
    plt.legend(loc = 'right', fontsize = 15)
    plt.title("Water Mass Transport Above The Sill Crest", size = 25, weight = 'bold')
    #fig.autofmt_xdate(rotation=50)
  
    #plt.show()
    plt.close()
    fig.savefig(fig_path + '/Qf' + format(i, '03d') + '.png', dpi = 150, bbox_inches='tight')